# Python On Resonance (PyOR)
## Everybody can simulate NMR

Author: Vineeth Thalakottoor @ ENS, Paris

Email: vineethfrancis.physics@gmail.com

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Tutorial 16 Redfield and Lindblad Master Equation

In this tutorial I will show Redfield and Lindblad Master Equation, their differences and how to implement them numercially. This tutorial is only for **two spin half system** and **HOMONUCLEAR** dipolar relaxation mechanism, and focused on numerical computation point of view. 

Reference 1 : Protein NMR Spectroscopy Principles and Practice, John Cavanagh, et. al. (Edition 1$^{st}$).

Reference 2 : A master equation for spin systems far from equilibrium, Bengs and Levitt, J.M.R, 310, 2020.

## Liouville-von Neumann equation

The Liouville-von Neumann equation,

$$
\frac{d}{dt} \rho = \frac{-i}{\hbar} [H_{0},\rho]
$$

has a solution given by, $\rho (t) = e^{i H_{0} t} \rho(0) e^{-i H_{0} t}$. From computation point of view, both $\rho$ (density matrix) and $H_{0}$ (Hamiltonian) are $4 \times 4$ matrices. We call the vector space where $\rho_{4 \times 4}$ lies Hilbert space.

What if we can write Liouville-von Neumann equation as

$$
\frac{d}{dt} \tilde{\rho} = \frac{-i}{\hbar} \hat{H}_{0} \tilde{\rho}
$$

where $\tilde{\rho}$ is a $16 \times 1$ matrix, $\hat{H}_{0}$ is a $16 \times 16$ matrix and has solution $\tilde{\rho} (t) = e^{-i \hat{H}_{0} t} \tilde{\rho}(0)$

For this let me introduce a small trick: Flat$(A \rho B)$ = $A \otimes B^{T}$ Flat($\rho$). When the function 'Flat()' act on a square matrix, $X_{n\times n}$ gives $X_{n^{2}\times 1}$ (please check yourself - I can upload a mathematica file for this). 

Lets apply this Liouville-von Neumann equation using identity matrix ($\mathbb{1}$),

$$
\frac{d}{dt} \mathbb{1} \rho \mathbb{1} = \frac{-i}{\hbar} [H_{0},\rho] = \frac{-i}{\hbar} (H_{0} \rho \mathbb{1} - \mathbb{1} \rho H_{0})
$$

then flatten the above equation we get,
$$
\frac{d}{dt} \tilde{\rho} = \frac{-i}{\hbar} (H_{0} \otimes \mathbb{1} - \mathbb{1} \otimes H_{0}^{T})\tilde{\rho}
$$

where $\hat{H}_{0} = H_{0} \otimes \mathbb{1} - \mathbb{1} \otimes H_{0}^{T}$ is called the Hamiltonian commutation superoperator. We call the vector space where $\rho_{16 \times 1}$ lies Liouville space

## Some definitions

1. let $X$ be a observable (example $H_{0}$, Hamiltonian of the system) in Hilbert sapce


2. $\rho$ is a density matrix (square matrix) which lies in Hilbert space. The basis of this space are {$|\alpha \alpha><\alpha \alpha|$, $|\alpha \alpha><\alpha \beta|$, ...}$_{16}$ or {$\frac{1}{2}E$, $\frac{1}{\sqrt{2}}I_{+}$,$\frac{1}{\sqrt{2}}S_{+}$,...,$I_{+}S_{+}$}$_{16}$

$$\rho = \begin{bmatrix} \rho_{11} & \rho_{12} & \rho_{13} &  \rho_{14} \\ \rho_{21} & \rho_{22} & \rho_{23} &  \rho_{24} \\ \rho_{31} & \rho_{32} & \rho_{33} &  \rho_{34} \\ \rho_{41} & \rho_{42} & \rho_{43} &  \rho_{44} \end{bmatrix}$$

3. $\tilde{\rho}$ is a vector (coloumn vector) which lies in Liouville space. 

$$\tilde{\rho} = \begin{bmatrix} \rho_{11} \\ \rho_{12} \\ \rho_{13} \\  \rho_{14} \\ \rho_{21} \\ \vdots \\ \rho_{44}\end{bmatrix}$$

4. Relation between $\rho$ and $\tilde{\rho}$ is given by $\tilde{\rho}$ = Flat($\mathbb{1} \rho \mathbb{1}$) = Flat($\rho$)


5. Superoperator is a operator when act on a vector ($\tilde{\rho}$) in Liouville space gives another vector which also lie in Liouville space.


6. Left Superoperator, $\hat{X_{L}}$ = $X \otimes \mathbb{1}$; corresponding to $X \rho$ in Hilbert space


7. Right Superoperator, $\hat{X_{R}}$ = $\mathbb{1} \otimes X^{T}$; corresponding to $\rho X$ in Hilbert space 


8. Commutation Superoperator, $\hat{X}$ = $X \otimes \mathbb{1} - \mathbb{1} \otimes X^{T}$; corresponding to $[X,\rho] = X \rho$ -$\rho X$ in Hilbert space


9. Anti Commutation Superoperator, corresponding to $\{X,\rho\} = X \rho$ + $\rho X$ is defiend by $X \otimes \mathbb{1} + \mathbb{1} \otimes X^{T}$ in Hilbert space


10. Double Commutation Superoperator, $\hat{X}\hat{Y}$ (matrix multiplication of $\hat{X}$ and $\hat{Y}$) corresponding to $[X,[Y,\rho]]$ in Hilbert space

We will make use these concepts below

## Redfield Master Equation in Hilbert Space

I am not going to the depth of the Redfield theory. We consider two spin half particles, $H_{0} = \omega_{I}I_{z} + \omega_{S}S_{z}$ is the time independent Hamiltonian of the system and $H_{1} = \sum_{q = -2}^{2} F_{2}^{q} A_{2}^{q}$ is the stochastic Hamiltonian (dipolar) causing the relaxation, where $F_{2}^{q}$ and $A_{2}^{q}$ are the spatial and spin part of the Hamiltonian. $A_{2}^{q}$ is the 2$^{nd}$ rank spherical tensors with q = -2, -1, 0, 1, 2 and can be written as linear combination of operator basis($O_{i=1-16}$), $A_{2}^{q} = \sum_{p} A_{2p}^{q} = \sum_{p} C_{p} O_{p}$. $O_{p}$ satisfies the relation $[H_{0},O_{i}] = \omega_{i} O_{i}$ where $O_{i}$ and $\omega_{i}$ eigen vectors and eigen frequencies of commutation superoperator ($\hat{H}$), $\hat{H} X = [H,X] = HX-XH$. The Redfield master equation is given by

$$
\frac{d}{dt} \rho = \frac{-i}{\hbar} [H_{0},\rho] + \sum_{q = -2}^{2} \sum_{p} S(\omega_{p}^{q}) [A_{2p}^{q},[(A_{2p}^{q})^{\dagger},\rho - \rho_{eq}]]
$$
Here density matrix, $\rho$ is the density matrix and $\rho_{eq}$ is the equlibrium density matrix. From computation point of view, we can consider $\rho$, $H_{0}$ and $A_{2p}^{q}$ as $4 \times 4$ matrices. $S(\omega)$ is the spectral density function proportional to $$\frac{2\tau}{1+\omega^{2}\tau^{2}}$$, $\tau$ is the correlation time.

For clarity: 

1. Zero Quantum, $\omega^{q=0} = 0$

$A_{2}^{0} = \frac{1}{2 \sqrt{6}}(4 I_{z}S_{z} - I_{+}S_{-} - I_{-}S_{+})$, where $A_{21}^{0} = \frac{4}{2 \sqrt{6}} I_{z}S_{z}$, $A_{22}^{0} = \frac{-1}{2 \sqrt{6}}4 I_{+}S_{-}$ and $A_{23}^{0} = \frac{-1}{2 \sqrt{6}}4 I_{-}S_{+}$

2. Single Quantum  $\omega^{q=\pm 1} = \pm \omega_{0}$ ($\omega_{0} = \omega_{I} \approx \omega_{S}$)

$A_{2}^{\pm1} = \mp\frac{1}{2} (I_{\pm}S_{z} + I_{z}S_{\pm}) $, where $A_{21}^{\pm1} = \mp\frac{1}{2}I_{\pm}S_{z} $, $A_{22}^{\pm1} = \mp\frac{1}{2}I_{z}S_{\pm} $

3. Double Quantum  $\omega^{q=\pm 2} = \pm 2 \omega_{0}$

$A_{2}^{\pm2} = \frac{1}{2} I_{\pm}S_{\pm} $, where $A_{21}^{\pm2} = \frac{1}{2}I_{\pm}S_{\pm} $

Please note that this is note the case for hetronuclear.

## Redfield Master Equation in Liouville Space

$$
\frac{d}{dt} \tilde{\rho} = \frac{-i}{\hbar} (\hat{H}_{0} + i \hat{R})\tilde{\rho}
$$

where $\tilde{\rho}$ represent vectorized or flattened (coloumn vector) density operator $\rho$. $\hat{H}_{0}$ is the Hamiltonian commutation superoperator and $\hat{R}$ is the relaxation superoperator and defined as:

$$
\hat{R} = \sum_{q = -2}^{2} \sum_{p} S(\omega_{p}^{q}) \hat{A}_{2p}^{q} (\hat{A}_{2p}^{q})^{\dagger}
$$

The solution of the master equation is given by (considering return to equlibrium):

$$
\tilde{\rho} (t) = \tilde{\rho}_{eq} + e^{-i(\hat{H}_{0} + i \hat{R})}(\tilde{\rho}(0)- \tilde{\rho}_{eq})
$$

## Lindblad Master Equation in Liouville Space

$$
\frac{d}{dt} \tilde{\rho} = \frac{-i}{\hbar} (\hat{H}_{0} + i \hat{R_{L}})\tilde{\rho}
$$

where $\tilde{\rho}$ represent vectorized or flattened (coloumn vector) density operator $\rho$. $\hat{H}_{0}$ is the Hamiltonian commutation superoperator and $\hat{R_L}$ is the relaxation superoperator and defined as:

$$
\hat{R}_{L} = \sum_{q = -2}^{2} \sum_{p} S_L(\omega_{p}^{q}) \hat{L}_{dis}[A_{2p}^{q}, (A_{2p}^{q})^{\dagger}]
$$

The solution of the master equation is given by :

$$
\tilde{\rho} (t) = e^{-i(\hat{H}_{0} + i \hat{R}_{L})} \tilde{\rho}(0)
$$

## Lindbladian Dissipator

$$
\hat{L}_{dis}[A,B] = A \otimes B^{T} - \frac{1}{2} (BA \otimes \mathbb{1} + \mathbb{1} \otimes A^{T} B^{T})
$$

Note that here $A$ and $B$ are operator basis of Hilbert space.

## Lindbladian Spectral density function

$$
S_L(\omega) = \frac{6}{5} b_{IS}^{2} \frac{2\tau}{1+\omega^{2}\tau^{2}} e^{-\frac{1}{2} \beta \hbar \omega} 
$$

Where $\beta = 1 / k_{B} T$, inverse spin temperature and $b_{IS}^{2}$ is the dipolar coupling constant.

## Any suggestion? write to me
If you see something is wrong or need more explanation please write to me. 

vineethfrancis.physics@gmail.com